In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re

In [ ]:
url_imdb = 'http://www.imdb.com/title/tt0451279/?ref_=nv_sr_1'
url_tomates = 'https://www.rottentomatoes.com/m/wonder_woman_2017'
url_filmow = 'https://filmow.com/mulher-maravilha-t48706/'

# IMDb

In [ ]:
# Get the page content and set up a new parser
response = requests.get(url_imdb)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

# Get title
title_imd = soup.find_all("h1", itemprop="name")[0].text

# Get rating
rating_imdb = soup.find_all("span", itemprop="ratingValue")[0].text
userRating_imdb = soup.find_all("span", itemprop="ratingCount")[0].text

# Get time
time_imdb = soup.find_all("time", itemprop="duration")[0].text

# Get genres
genres = soup.find_all("span", itemprop="genre", class_="itemprop")
g_imdb = ""
for genre in genres:
    g_imdb = g_imdb + genre.text + ", "

# Rotten tomates

In [ ]:
# Get the page content and set up a new parser
response = requests.get(url_tomates)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

### TOMATOMETER
# Get rating
rating_tmt = soup.find_all("span", class_="meter-value superPageFontColor")[0].text
divs_tmt = soup.find_all("div", id="scoreStats")[0].find_all("div", class_="superPageFontColor")
critic_tmt = soup.find_all("p", class_="critic_consensus superPageFontColor")[0]

### AUDIENCE SCORE
# Get audience
audience_tmt = soup.find_all("span", class_="superPageFontColor")[0].text
divs_tmt_audience = soup.find_all("div", class_="audience-info hidden-xs superPageFontColor")[0].find_all("div")

# Synopsis
movie_synopsis_tmt = soup.find_all("div", id="movieSynopsis")[0]

# Filmow #To-do

In [ ]:
# Get the page content and set up a new parser
response = requests.get(url_filmow)
content = response.content
soup = BeautifulSoup(content, 'html.parser')

# Get cast
rating_filmow = soup.find_all("span", itemprop="ratingValue")[0].text
ratingCount_filmow = soup.find_all("span", itemprop="ratingCount")[0].text

# Get cast
lis_filmow = soup.find_all("ul", id="casting")[0].find_all("li")

In [ ]:
# IMDB
print("Title: " + title_imd.strip())
print("Rating (IMDB): " + rating_imdb + "/10 (" + userRating_imdb + " users rating)")
print("Time: " + time_imdb.strip())
print("Genres: " + g_imdb[0:-2])

# Rotten Tomates
print("\n----- TOMATOMETER -----")
print("Rating: " + rating_tmt)
for div in divs_tmt:
    if(len(div.find_all("span")) == 2):
        print(div.find_all("span")[0].text + div.find_all("span")[1].text)
    else:
        print(div.text.strip())
print(critic_tmt.text_tmt)
print("\n----- AUDIENCE SCORE -----")
print("Audience rating: " + audience_tmt)
for div in divs_tmt_audience:
    print(div.text.strip())
print("\n----- Movie Info -----")
print("Synopsis: " + movie_synopsis_tmt.text)

# Filmow
print("Rating (Filmow): " + rating_filmow + " (Based on " + ratingCount_filmow + " votes)")
print("\n----- Casting -----")
for li in lis_filmow:
    actor = li.find_all("span", itemprop="name")[0].text
    paper = li.find_all("em")
    if paper:
        print(actor + " - " + paper[0].text)
    else:
        print(actor)

# Getting movie theater schedule data

In [ ]:
# Add the lib package to the system path, so that we can include modules from there.

import os
import sys

module_path = os.path.abspath(os.path.join('./'))

if module_path not in sys.path:
    sys.path.append(module_path)
    
from lib.movie_theater_schedule.cinepolis_schedule import CinepolisSchedule
from lib.movie_theater_schedule.cinemark_schedule import CinemarkSchedule

## Cinépolis

In [ ]:
def generate_cinepolis_soup(cod_cinema, cod_claquete):
    body = {
        'cod_cinema'   : cod_cinema,
        'cod_claquete' : cod_claquete,
        'cod_horario'  : time.strftime("%Y-%m-%d"),
        'cod_filme'    : '0'
    }
    page = requests.post('http://www.cinepolis.com.br/programacao/ajax/ajax.conteudo_horarios.php', data = body)
    return BeautifulSoup(page.content, 'html.parser')

In [ ]:
def get_cinepolis_schedule_data(soup, movie_theater):
    table = soup.find("table", { "class" : "tabelahorarios" })
    rows = table.find_all('tr')
    
    has_movie = lambda row : row.find(attrs={'data-order': re.compile('\w+')})

    entries = [row for row in rows if has_movie(row)]

    rows_list = []

    for entry in entries:
        title = entry.find_all(href=re.compile('http://www.cinepolis.com.br/filmes/filme.php'))[0].text

        room = entry.find_all('td')[0].text

        tags = []
        if(entry.find("a", { "class" : "icovip" })):
            tags.append('VIP')
        if(entry.find("a", { "class" : "icomacroxe" })):
            tags.append('MacroXE')
        if(entry.find("a", { "class" : "ico3d" })):
            tags.append('3D')
        tags = ' / '.join(tags)

        content_rating = entry.find_all('td')[2].find('img').get('alt')

        category = entry.find("td", { "class" : "horarios" }).find('span').get('aria-label')

        schedules_tags = entry.find("td", { "class" : "horarios" }).select("span + span, a")
        schedules = ' / '.join([s.text for s in schedules_tags])

        row_dict = {
            'room': room,
            'title': title,
            'tags': tags,
            'content_rating': content_rating,
            'category': category,
            'schedules': schedules,
            'movie_theater': movie_theater
        }
        rows_list.append(row_dict)

    return pd.DataFrame(rows_list)

### Cinépolis Natal Shopping

In [ ]:
cinepolis_natal_soup = generate_cinepolis_soup('31', '769')
cinepolis_natal_schedule = CinepolisSchedule(cinepolis_natal_soup, 'Cinépolis Natal Shopping')
cinepolis_natal_schedule_df = cinepolis_natal_schedule.get_dataframe()

### Cinépolis Partage Norte Shopping Natal

In [ ]:
cinepolis_norte_soup = generate_cinepolis_soup('33', '770')
cinepolis_norte_schedule = CinepolisSchedule(cinepolis_norte_soup, 'Cinépolis Partage Norte Shopping Natal')
cinepolis_norte_schedule_df = cinepolis_norte_schedule.get_dataframe()

## Cinemark

In [ ]:
response = requests.get('https://www.cinemark.com.br/natal/cinemas')
cinemark_soup = BeautifulSoup(response.content, 'html.parser')
cinemark_schedule = CinemarkSchedule(cinemark_soup, 'Cinemark Midway Mall Natal')
cinemark_schedule_df = cinemark_schedule.get_dataframe()

## Movie theaters schedule data

In [ ]:
frames = [cinepolis_natal_schedule_df, cinepolis_norte_schedule_df, cinemark_schedule_df]
movie_theaters_schedule_data = pd.concat(frames)

In [ ]:
movie_theaters_schedule_data.sample()